# 라이브러리 임포트

In [10]:
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver

from tqdm import tqdm
from app_store_scraper import AppStore

import time
import pandas as pd
import numpy as np
import random
from datetime import datetime


# 수집기간 및 저장경로 지정

In [ ]:
start_date = datetime(2024, 1, 1)

## PlayStore 리뷰 수집

In [25]:
from google_play_scraper import Sort, reviews_all

In [39]:
pay_app = {'kakaopay': 1464496236, 'naverpay': 1554807824, 'toss': 839333328}

app_reviews = reviews_all('com.kakaopay.app', sleep_milliseconds=10, country = 'kr', sort = Sort.NEWEST)

reviews_list = [
    {'review_id': review['reviewId'],
     'review': review['content'],
     'score': review['score'],
     'like': review['thumbsUpCount'],
     'date': review['at'].strftime('%Y-%m-%d %H:%M:%S')}  # 날짜 변환
    for review in app_reviews
]

reviews_df = pd.DataFrame(reviews_list)

In [40]:
reviews_df

,review_id,review,score,like,date
0,f78bec61-7da4-4eca-906b-59b3d676ada9,Really sucks.. Forcing everything,1,0,2025-01-17 09:03:43
1,da2fa32b-6034-43ca-8874-e63002bcb2ba,Good,5,0,2024-12-25 01:19:21
2,119504c3-66a9-4550-9584-9546a095c6b5,여어로 주세요,3,0,2024-12-10 18:37:58
3,012f58cc-71d0-44a2-a6ba-094906ad5864,편하고 좋습니다.,5,0,2024-12-05 09:30:52
4,eab71c1c-2654-4152-9994-470c39e348c4,"Because I left the country, I can no longer ac...",1,4,2024-09-08 12:42:05
...,...,...,...,...,...
100,d4f75224-2c15-4d9c-859e-752be24239f7,5% 이벤트 신청했는데 사용중인 기기는 업그레이드를 지원하지 않는다고 뜨네요. 언제...,3,0,2020-03-05 17:13:22
101,aa54a197-c02e-4b44-95a9-a74d74f28821,증권사 잔액/내역 조회 얼른 힘 좀 내주십셔,3,3,2019-12-23 01:45:01
102,d586e25d-b02e-4f44-bbbc-ca1fb70256fa,Foreigners cannot use this app,1,15,2019-08-23 19:10:45
103,b11ea143-fea4-4abc-8675-32a912bcdf35,this app is not intuitive though simple and in...,1,9,2019-08-07 21:12:41


## AppStore 리뷰 수집

In [20]:
def Extract_reviews(app_name:str, app_id:int): 
  pay = AppStore(country='kr', app_name=app_name, app_id=app_id)
  pay.review(after = start_date)
  pay_reviews = pd.DataFrame(pay.reviews).sort_values('date', ascending=False).reset_index(drop=True)
  pay_reviews['pay'] = app_name
  return pay_reviews

In [21]:
pay_app = {'kakaopay': 1464496236, 'naverpay': 1554807824, 'toss': 839333328}

app_store_reviews = pd.DataFrame()
for i in tqdm(pay_app):
  pay_reviews = Extract_reviews(i, pay_app[i])
  app_store_reviews= pd.concat([app_store_reviews, pay_reviews], axis=0)

print('Done..')

  0%|          | 0/3 [00:00<?, ?it/s]2025-01-27 17:32:02,333 [INFO] Base - Initialised: AppStore('kr', 'kakaopay', 1464496236)
2025-01-27 17:32:02,333 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/kr/app/kakaopay/id1464496236
2025-01-27 17:32:06,407 [INFO] Base - [id:1464496236] Fetched 260 reviews (260 fetched in total)
 33%|███▎      | 1/3 [00:04<00:08,  4.20s/it]2025-01-27 17:32:06,534 [INFO] Base - Initialised: AppStore('kr', 'naverpay', 1554807824)
2025-01-27 17:32:06,534 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/kr/app/naverpay/id1554807824
2025-01-27 17:32:08,685 [INFO] Base - [id:1554807824] Fetched 280 reviews (280 fetched in total)
 67%|██████▋   | 2/3 [00:06<00:03,  3.07s/it]2025-01-27 17:32:08,807 [INFO] Base - Initialised: AppStore('kr', 'toss', 839333328)
2025-01-27 17:32:08,807 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/kr/app/toss/id839333328
2025-01-27 17:32:27,518 [ERROR] Base - Something went wrong: H

KeyError: 'date'

In [19]:
app_store_reviews

,date,developerResponse,review,rating,isEdited,title,userName,pay
0,2023-04-12 11:11:59,"{'id': 35986362, 'body': '안녕하세요, 카카오페이 앱을 이용해주...",주식때문에 하루에도 몇번? 몇십번을 열어보는 어플인데 사실 불편한 점이나 버퍼링이 ...,5,False,이번 업데이트 만족,yuni3yuni,kakaopay
1,2023-07-25 07:30:55,"{'id': 38034211, 'body': '안녕하세요, 카카오페이 서비스를 이용...",카카오페이는 결제나 송금만 이용했었는데 주식 되고 나서 접근성도 좋고 주식 알림 걸...,5,False,주식 관련 알림과 정보 주는거 너무 좋아요!,뉸뉴냔냐냐히,kakaopay
2,2023-05-17 00:17:46,"{'id': 36608464, 'body': '안녕하세요, 와플 왕팬님. 서비스 이...","[주식 선물하기] > [누구에게 주식을 선물할까요?]를 누르면 ""일시적인 오류가 발...",5,False,주식 선물하기 기능 오류,와플 왕팬,kakaopay
3,2021-09-27 16:00:06,"{'id': 25381057, 'body': '안녕하세요, j_jay_님. 카카오페...","이번 개편 버전 정말 대박입니다 😍 홈도 필요한 기능으로 설정할 수 있고 팁, 혜택...",5,False,금융 플랫폼 끝판왕이네요!,j_jay_,kakaopay
4,2021-06-22 20:10:42,"{'id': 23553595, 'body': '안녕하세요, bwidhvrc님. 카카...",카카오페이 결제화면에서 기본은 카카오페이머니로 돼있어서 결제수단을 카드로 변경해야할...,3,False,결제칸에서 카드 선택할때 위아래로 쓸어넘겨서 바꿀수있게 해주세요,bwidhvrc,kakaopay
...,...,...,...,...,...,...,...,...
1635,2019-01-19 13:37:43,"{'id': 6922528, 'body': '안녕하세요, 수돌짱짱맨 님. 토스팀입니...",토스 편해서 자주 애용하고 있었는데 착오송금건에 대해서 너무 자기일아니라고 편하신것...,2,False,리뷰 처음 씁니다,수돌짱짱맨,toss
1636,2018-12-28 11:01:13,"{'id': 6617124, 'body': '안녕하세요 토스 입니다. 서비스 이용...",정말 빠르고 좋지만 제 개인적인 의견은 좀 불편해진 것 같아요. 들어와서 사용법을 ...,5,False,짱이에요,야야우너,toss
1637,2018-12-23 08:44:32,"{'id': 6616475, 'body': '안녕하세요, 토스입니다. 우선 고객님께...",서비스 점검시간이랬나 아무튼 그 시간에 입금이 불가능해서 자동이체 해준다고 예약하라...,5,False,토스 자동이체,래로라라,toss
1638,2018-11-16 09:17:23,"{'id': 5986451, 'body': '안녕하세요, 토스입니다. 현재 토스는...","보안 관리도 못 하는 주제에 매번 짜증나게 보안카드, 공인인증서 요구하는 은행 애플...",5,False,계좌 관리의 혁명,ㅁ허ㅏㅇ서ㅓ,toss


In [ ]:
# 크롬 드라이버 제어
driver = webdriver.Chrome()